In [1]:
import numpy as np
from Robot import UR5Arm
from fwdkin import fwdkin
import math
from path_planning import path_planner
from Arm_Lib import Arm_Device
from utils import rot
Arm = Arm_Device()


In [2]:
ex = np.array([1,0,0])
ey = np.array([0,1,0])
ez = np.array([0,0,1])
l0 = 0.061 # base to servo 1
l1 = 0.0435 # servo 1 to servo 2
l2 = 0.08285 # servo 2 to servo 3
l3 = 0.08285 # servo 3 to servo 4
l4 = 0.07385 # servo 4 to servo 5
l5 = 0.05457 # servo 5 to gripper
P01 = ( l0 + l1 ) * ez 
P12 = np.zeros (3) # translation between 1 and 2 frame in 1 frame
P23 = l2 * ex # translation between 2 and 3 frame in 2 frame
P34 = - l3 * ez # translation between 3 and 4 frame in 3 frame
P45 = np.zeros (3) # translation between 4 and 5 frame in 4 frame
P5T = -( l4 + l5 ) * ex 

P = np.array([P01, P12, P23, P34, P45, P5T]).T
H = np.array([ez, -ey, -ey, -ey, -ex]).T
print(P,H)
limits = None 
dofbot = UR5Arm(P, H, limits)

[[ 0.       0.       0.08285 -0.       0.      -0.12842]
 [ 0.       0.       0.      -0.       0.      -0.     ]
 [ 0.1045   0.       0.      -0.08285  0.      -0.     ]] [[ 0  0  0  0 -1]
 [ 0 -1 -1 -1  0]
 [ 1  0  0  0  0]]


In [3]:
yellow = np.array([62, 31, 31, 90, 90])[None].T*math.pi/180
blue = np.array(  [44, 66, 20, 28, 90])[None].T*math.pi/180
green = np.array([136, 70, 20, 28, 90])[None].T*math.pi/180
red = np.array(   [115, 28, 45, 75, 90])[None].T*math.pi/180

In [4]:
time = 800
N = 20
Kp = 0.0000001
Ki = 0.00000001
Kd = 0.00000001

In [5]:
home = np.array([90, 90, 90, 90, 90])[None].T*math.pi/180
R_home, P_home = fwdkin(dofbot, home)

In [6]:
import time as ttime

In [7]:
# red
R_red, P_red = fwdkin(dofbot, red)
print(f"Red fwdkin: {R_red, P_red}")
path_planner(Arm, dofbot, N, R_home, P_home, R_red, P_red, Kp, Ki, Kd, 90, time, False)
ttime.sleep(1)
# read all angles
angles_nopid = []
for i in range(1,6):
    angles_nopid.append(Arm.Arm_serial_servo_read(i))
angles_nopid = np.array(angles_nopid)[None].T*math.pi/180
# run fwd kin

R_nopid, P_nopid = fwdkin(dofbot, angles_nopid)
print("Red, No PID")
print("Error:")
print(f"P_err: {P_red - P_nopid}")
print(f"R_err: {R_nopid @ R_red.T - np.eye(3)}")

ttime.sleep(4)





Red fwdkin: (array([[ 3.58400612e-01, -2.23953558e-01, -9.06307787e-01],
       [-7.68592593e-01,  4.80269956e-01, -4.22618262e-01],
       [ 5.29919264e-01,  8.48048096e-01, -9.41522522e-17]]), array([-0.11042525,  0.23680771,  0.05112049]))
[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-5.93641222e-08  1.31662043e-08 -3.82107213e-09  2.14878152e-05
  9.68848149e-05 -7.24935068e-07]
Inverse Kinematics Converged after 166 iterations
q_0: [1.35254194 1.57309424 1.56500276 1.57429191 1.78905071]
[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-1.20713820e-05 -2.28424552e-05 -2.53413447e-05  2.54765997e-05
 -3.29432314e-05 -9.43710350e-05]
Inverse Kinematics Converged after 97 iterations
q_dest: [2.00709355 0.48887139 0.78389044 1.31030486 1.57077793]
[ 77.49494458  90.13166098 

In [9]:
path_planner(Arm, dofbot, N, R_home, P_home, R_red, P_red, Kp, Ki, Kd, 90, time, True)
ttime.sleep(1)
# read all angles
angles_pid = []
for i in range(1,6):
    angles_pid.append(Arm.Arm_serial_servo_read(i))
angles_pid = np.array(angles_pid)[None].T*math.pi/180
# run fwd kin
R_pid, P_pid = fwdkin(dofbot, angles_pid)
print("Red, PID")
print("Error:")
print(f"P_err: {P_red - P_pid}")
print(f"R_err: {R_pid @ R_red.T - np.eye(3)}")

[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-5.93641222e-08  1.31662043e-08 -3.82107213e-09  2.14878152e-05
  9.68848149e-05 -7.24935068e-07]
Inverse Kinematics Converged after 166 iterations
q_0: [1.35254194 1.57309424 1.56500276 1.57429191 1.78905071]
[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-1.20713820e-05 -2.28424552e-05 -2.53413447e-05  2.54765997e-05
 -3.29432314e-05 -9.43710350e-05]
Inverse Kinematics Converged after 97 iterations
q_dest: [2.00709355 0.48887139 0.78389044 1.31030486 1.57077793]
Path following errors:
step 1: [-11.50505542   0.13166098  -0.33194666   0.2002822   12.5050552 ]
step 2: [-9.53121095 -3.13788605 -2.68744346 -0.59578853  8.84683893]
step 3: [-2.55736648 -6.40743307 -5.04294026 -1.39185926  4.18862265]
step 4: [ 2.41647798 -8.67698009